In [1]:
import pandas as pd
import numpy as np
import paddle

In [2]:
data = pd.read_csv("all_movies_with_id.csv")

D:\ANACONDA\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
comment = data["Comment"][:1000]
star = data["Star"][:1000]
data=None

In [4]:
label = []
for i in star:
    if i>=4:
        label.append(2)
    elif i<=2:
        label.append(0)
    else:
        label.append(1)

In [5]:
import jieba
from collections import Counter

In [6]:
cnt = Counter()
for sentence in comment:
    words = jieba.cut(sentence,cut_all=True)
    cnt+=Counter(words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zhang\AppData\Local\Temp\jieba.cache
Loading model cost 0.671 seconds.
Prefix dict has been built successfully.


In [7]:

MAX_LEN=30

In [8]:
count = 1
word_to_id_dict = {}
for i in cnt:
    word_to_id_dict[i]=count
    count+=1
word_to_id_dict["unk"] = count
word_to_id_dict["<pad>"] = 0

id_to_word_dict ={j:i for i,j in zip(word_to_id_dict.keys(),word_to_id_dict.values())}
word_size = len(id_to_word_dict)

dict_size = len(id_to_word_dict)

In [9]:
def get_data(word_to_id_dict,comment,label):
    train_inputs =[]
    test_inputs=[]
    for i,k in zip(comment[:-100],label[:-100]):
        temp = []
        for j in jieba.cut(i):
            temp.append(word_to_id_dict.get(j,word_to_id_dict["unk"]))
        if len(temp)>MAX_LEN:
            temp = temp[:MAX_LEN]
        elif len(temp)<MAX_LEN:
            temp = temp+[0]*(MAX_LEN-len(temp))
        train_inputs.append((temp,k))
    for i,k in zip(comment[-100:],label[-100:]):
        temp = []
        for j in jieba.cut(i):
            temp.append(word_to_id_dict.get(j,word_to_id_dict["unk"]))
        if len(temp)>MAX_LEN:
            temp = temp[:MAX_LEN]
        elif len(temp)<MAX_LEN:
            temp = temp+[0]*(MAX_LEN-len(temp))
        test_inputs.append((temp,k))
    
    return train_inputs,test_inputs


train_data,test_data = get_data(word_to_id_dict,comment,label)

    
        
            
            

In [18]:
class MyTextDataset(paddle.io.Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data
        self.length = len(data)
    def __getitem__(self,index):
        input_x = self.data[index][0]
        label = self.data[index][1]
        return np.array(input_x),label
#         return input_x,label
    def __len__(self):
        return self.length

In [21]:
mydata_train = MyTextDataset(train_data)
mydata_test = MyTextDataset(test_data)


train_dataload = paddle.io.DataLoader(mydata_train,batch_size=64,shuffle=True,drop_last=False)
test_dataload =  paddle.io.DataLoader(mydata_test,batch_size=64,shuffle=True,drop_last=False)

In [23]:
class MyModel(paddle.nn.Layer):
    def __init__(self,dict_size):
        super().__init__()
        
        self.embedding = paddle.nn.Embedding(dict_size,128)
        self.rnn = paddle.nn.LSTM(input_size=128,hidden_size=128,num_layers=1,dropout=0.2)
        self.linear = paddle.nn.Linear(128,3)
    def forward(self,x):
        
        x= self.embedding(x)
        out,_ = self.rnn(x)
        out = paddle.mean(out,1)
        out = self.linear(out)
        return out
        

In [24]:
model = MyModel(dict_size)


In [30]:
opt = paddle.optimizer.Adam(learning_rate=0.001,parameters=model.parameters())
loss_fn = paddle.nn.CrossEntropyLoss()

In [41]:
for input_x,label in train_dataload:
    
    label = label.astype(paddle.int64)
    out = model(input_x)
    
    loss = loss_fn(out,label)
    print(loss.item())
    loss.backward()
    opt.step()
    opt.clear_grad()

0.8230083584785461
0.7659845352172852
0.7596207857131958
0.7198231220245361
0.7326130867004395
0.7270877361297607
0.7580116391181946
0.7230734825134277
0.8267865180969238
0.7633288502693176
0.6290515661239624
0.75119948387146
0.6527823209762573
0.6014413237571716
0.7892528176307678


In [54]:
def precess(txt,word_to_id_dict):
    temp = []
    for i in jieba.cut(txt):
        temp.append(word_to_id_dict.get(i,word_to_id_dict["unk"]))
    
    
    inputs = paddle.to_tensor([temp])
    return inputs

inputs = "我非常的喜欢中国，我太爱中国了，中国是我的家"

out = model(precess(inputs,word_to_id_dict))
 


        
    
    

In [62]:
pre = paddle.argmax(out,-1).item()
if pre==0:
    print("消极")
elif pre==1:
    print("中立")
else:
    print("积极")


中立
